In [1]:
import functools
import os
import pickle
import random

import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels.formula.api as smf
from scipy.special import gamma


@functools.wraps(smf.ols)
def lm(*args, **kwargs):
    return smf.ols(*args, **kwargs).fit()

In [2]:
os.chdir("../data")

In [3]:
recommendee = "Fro116"
neighborhood_size = 64
confidence_interval = 0.99
full_neighborhoods = False
perform_regression = True
nonneg_corrs=True

In [4]:
anime = pd.read_csv("AnimeList.csv")
anime = anime[["anime_id", "title", "type"]]

In [5]:
df = pd.read_csv("UserAnimeList.csv")

In [6]:
len(df["username"].unique()), len(df["anime_id"].unique()), len(df) / (
    len(df["username"].unique()) * len(df["anime_id"].unique())
), len(df)

(283045, 14478, 0.01954064606703893, 80076112)

In [7]:
filtered_df = df[["username", "anime_id", "my_score"]].loc[lambda x: x["my_score"] != 0]

In [8]:
# add additional user anime-lists
extraUsers = pickle.load(open("user_profiles/ExtraUserAnimeLists.pkl", "rb"))
filtered_df = filtered_df.loc[lambda x: ~x["username"].isin(extraUsers.username)]
filtered_df = pd.concat([filtered_df, extraUsers], ignore_index=True)

In [9]:
average_rating = filtered_df["my_score"].mean()
user_bias = (
    pd.DataFrame(filtered_df.groupby("username")["my_score"].mean()).rename(
        {"my_score": "user_bias"}, axis=1
    )
    - average_rating
)
anime_bias = (
    pd.DataFrame(filtered_df.groupby("anime_id")["my_score"].mean()).rename(
        {"my_score": "anime_bias"}, axis=1
    )
    - average_rating
)

In [10]:
filtered_df = filtered_df.merge(anime_bias, on=["anime_id"]).merge(
    user_bias, on=["username"]
)
filtered_df["normalized_score"] = (
    filtered_df["my_score"]
    - filtered_df["anime_bias"]
    - filtered_df["user_bias"]
    - average_rating
)
filtered_df = filtered_df.set_index("username")
filtered_df = filtered_df.dropna()

In [11]:
filtered_df

,anime_id,my_score,anime_bias,user_bias,normalized_score
username,,,,,
karthiga,21,9,0.960564,-0.059898,0.605474
karthiga,59,7,0.040203,-0.059898,-0.474165
karthiga,74,7,0.316282,-0.059898,-0.750245
karthiga,120,7,0.309858,-0.059898,-0.743821
karthiga,178,7,-0.227339,-0.059898,-0.206624
...,...,...,...,...,...
temptemptemp,10040,6,-1.636717,-1.493860,1.636717
cinnamoroller,12963,10,-0.798860,2.506140,0.798860
inactiveX,5143,7,-0.652951,-0.493860,0.652951


In [12]:
all_corrs = pickle.load(open("item_correlations/correlations.pkl", "rb"))

In [13]:
if nonneg_corrs:
    all_corrs["similarity"] = all_corrs["corr"]
else:
    all_corrs["similarity"] = all_corrs["corr"].abs()
all_corrs = all_corrs.dropna()

In [14]:
corrs = all_corrs.loc[
    lambda x: x.index.get_level_values("anime_id_x")
    != x.index.get_level_values("anime_id_y")
]

In [15]:
if full_neighborhoods:
    corrs = corrs.groupby("anime_id_x").apply(lambda x: x.sort_values(by="similarity"))
else:
    corrs = corrs.groupby("anime_id_x").apply(
        lambda x: x.sort_values(by="similarity")[-neighborhood_size:]
    )

In [16]:
corrs.index = corrs.index.droplevel()

In [17]:
corrs

corr  corr_var   size  similarity
anime_id_x anime_id_y                                       
1          24687       0.084376  0.000172   5745    0.084376
           239         0.085037  0.000085  11599    0.085037
           440         0.085474  0.000131   7531    0.085474
           2236        0.088889  0.000030  33034    0.088889
           875         0.089690  0.000192   5127    0.089690
...                         ...       ...    ...         ...
37908      28913       0.207284  0.915913      3    0.207284
           28915       0.226825  0.899748      3    0.226825
           28911       0.235301  0.892332      3    0.235301
           37124       0.267396  0.862111      3    0.267396
           37577       0.555375  0.478253      3    0.555375

[871837 rows x 4 columns]

In [18]:
score = filtered_df.loc[recommendee].merge(
    corrs.reset_index("anime_id_x"), left_on="anime_id", right_on="anime_id_y"
)

user_var = (
    pd.DataFrame(filtered_df.groupby("username")["normalized_score"].var())
    .rename({"normalized_score": "user_var"}, axis=1)
    .dropna()
)
score["user_var"] = user_var.loc[recommendee].squeeze()

anime_var = (
    pd.DataFrame(filtered_df.groupby("anime_id")["normalized_score"].var())
    .rename({"normalized_score": "anime_var"}, axis=1)
    .dropna()
)
score = score.merge(anime_var, on="anime_id")

score = score.drop("anime_id", axis=1).rename({"anime_id_x": "anime_id"}, axis=1)

if full_neighborhoods:
    score = (
        score.groupby("anime_id")
        .apply(lambda x: x.sort_values(by="similarity")[-neighborhood_size:])
        .reset_index(drop=True)
    )

In [19]:
score.head()

,my_score,anime_bias,user_bias,normalized_score,anime_id,corr,corr_var,size,similarity,user_var,anime_var
0,7,0.219362,-1.120204,0.406982,45,0.055748,0.000104,9546,0.055748,2.738753,1.630605
1,7,0.219362,-1.120204,0.406982,123,0.048346,0.000212,4696,0.048346,2.738753,1.630605
2,7,0.219362,-1.120204,0.406982,154,0.034332,0.000147,6792,0.034332,2.738753,1.630605
3,7,0.219362,-1.120204,0.406982,189,0.048751,0.000113,8816,0.048751,2.738753,1.630605
4,7,0.219362,-1.120204,0.406982,223,0.075094,0.000086,11440,0.075094,2.738753,1.630605


In [20]:
deltas = score.groupby("anime_id").apply(
    lambda x: np.dot(x["normalized_score"], x["corr"]) / x["corr"].abs().sum()
)
weights = score.groupby("anime_id").apply(lambda x: x["corr"].abs().sum())
counts = score.groupby("anime_id").size()

In [21]:
# The following formulae are used to compute the variance of the delta. Delta
# is a weighted sum of the form δ = Σ(s_i * w_i) / (Σw_i), where s_i is
# a vector scores for item i and w_i is the weight for item_i.
#
# By linearity, it suffices to compute (s_i * w_i) / (Σw_i). We assume that
# Var(s_i) is the same as the variance over the vector s_i (i.e. over
# all users s_i has rated). We treat w_i as a random variable with mean w_i
# and variance corr['corr_var']
#
# The variance for (w_i) / (Σw_i) can be estimated by doing a Taylor Approximation.
# See equation 20 of https://www.stat.cmu.edu/~hseltman/files/ratio.pdf. The
# formula for the ratio of two correlated variables R,S is
# Var(R/S) = E[R]^2/E[S]^2(Var[R]/E[R]^2 - 2Cov(R,S)/(E[R]E[S]) + Var[S]/E[S]^2)
#
# Lastly we take the product distribution of s_i and (w_i) / (Σw_i).
def correction_factor(x):
    return (
        1
        + x["corr_var"] / (x["corr"] ** 2)
        - 2 * x["corr_var"] / (x["corr"].abs().sum() * x["corr"].abs())
        + x["corr_var"].sum() / (x["corr"].abs().sum() ** 2)
    )


delta_var = score.groupby("anime_id").apply(
    lambda x: np.sum(x["user_var"] * x["corr"] ** 2 * correction_factor(x))
    / (x["corr"].abs().sum() ** 2)
)

# if the var < 0, then the ratio distribution approximation failed,
# usually because sample size is too small
delta_var.loc[lambda x: x < 0] = np.inf

# Apply a bessel correction to unbias the variance
average_weight = corrs.groupby("anime_id_x").apply(lambda x: x["corr"].abs().mean())
effective_sample_size = weights / average_weight
delta_var.loc[effective_sample_size <= 1] = np.inf
delta_var.loc[effective_sample_size > 1] *= effective_sample_size / (effective_sample_size - 1)

In [22]:
pred_df = pd.DataFrame()
pred_df["delta"] = deltas
pred_df["weight"] = weights
pred_df["counts"] = counts
pred_df["delta_sem"] = np.sqrt(delta_var)

# # Unbias the standard deviation estimate
# # See https://en.wikipedia.org/wiki/Unbiased_estimation_of_standard_deviation#Results_for_the_normal_distribution
# def standard_deviation_bias(n):
#     if n < 1:
#         return np.inf
#     if gamma((n - 1) / 2) == np.inf:
#         return 1
#     return np.sqrt(2 / (n - 1)) * gamma(n / 2) / gamma((n - 1) / 2)
# pred_df["delta_sem"] /= effective_sample_size.apply(standard_deviation_bias)

pred_df["blp"] = anime_bias + user_bias.loc[recommendee].squeeze() + average_rating
pred_df = pred_df.dropna()

recomendee_seen_shows = filtered_df.loc[recommendee].merge(pred_df, on=["anime_id"])
recomendee_seen_shows["target"] = (
    recomendee_seen_shows["my_score"] - recomendee_seen_shows["blp"]
)
if perform_regression:

    model = lm("target ~ delta + 0", recomendee_seen_shows)
    pred_df["score"] = model.predict(pred_df) + pred_df["blp"]
    pred_df["sem"] = np.sqrt(
        (
            (pred_df["delta_sem"] ** 2 + pred_df["delta"] ** 2)
            * (model.bse["delta"] ** 2 + model.params["delta"] ** 2)
        )
        - pred_df["delta"] ** 2 * model.params["delta"] ** 2
    )
else:
    pred_df["score"] = pred_df["delta"] + pred_df["blp"]
    pred_df["sem"] = pred_df["delta_sem"]


zscore = st.norm.ppf(1 - (1 - confidence_interval) / 2)
pred_df["score_lower_bound"] = pred_df["score"] - pred_df["sem"] * zscore
pred_df["score_upper_bound"] = pred_df["score"] + pred_df["sem"] * zscore

pred_df = pred_df.merge(anime, on="anime_id")
pred_df = pred_df.set_index("anime_id")

In [23]:
# confirm that setting blp = 1 is reasonable
print(lm("my_score ~ delta + blp + 0", recomendee_seen_shows).summary())

                                 OLS Regression Results                                
Dep. Variable:               my_score   R-squared (uncentered):                   0.948
Model:                            OLS   Adj. R-squared (uncentered):              0.948
Method:                 Least Squares   F-statistic:                              3028.
Date:                Thu, 13 May 2021   Prob (F-statistic):                   1.34e-212
Time:                        01:33:00   Log-Likelihood:                         -606.31
No. Observations:                 331   AIC:                                      1217.
Df Residuals:                     329   BIC:                                      1224.
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [24]:
# confirm that the top shows are ones that the user rates highly
pred_df.loc[lambda x: x["delta"] > 0].sort_values(
    by="score_lower_bound", ascending=False
)[:20]

,delta,weight,counts,delta_sem,blp,score,sem,score_lower_bound,score_upper_bound,title,type
anime_id,,,,,,,,,,,
34376,2.374423,1.499612,8,0.764762,6.999956,9.214976,0.755145,7.269853,11.160100,Yuuki Yuuna wa Yuusha de Aru: Washio Sumi no S...,Movie
34375,2.877506,1.293363,5,0.859110,6.725005,9.409334,0.855042,7.206892,11.611776,Yuuki Yuuna wa Yuusha de Aru: Washio Sumi no S...,Movie
3297,0.521180,5.067419,29,0.401371,7.557860,8.044052,0.380073,7.065048,9.023055,Aria The Origination,TV
34374,2.892285,1.311842,5,0.857929,6.402822,9.100938,0.854489,6.899920,11.301956,Yuuki Yuuna wa Yuusha de Aru: Washio Sumi no S...,Movie
962,0.818337,4.994643,26,0.435317,7.169297,7.932696,0.416379,6.860176,9.005216,Aria The Natural,TV
3784,0.302364,3.627105,34,0.386894,7.452614,7.734680,0.364195,6.796575,8.672785,Evangelion: 2.0 You Can (Not) Advance,Movie
2563,0.949164,4.750159,27,0.440205,6.983375,7.868819,0.423574,6.777764,8.959874,Aria The OVA: Arietta,OVA
397,1.530507,3.318693,20,0.600481,6.851996,8.279755,0.583443,6.776907,9.782604,Seikai no Senki II,TV
1729,1.031880,1.245211,20,0.458573,6.945361,7.907968,0.442219,6.768887,9.047048,Maria-sama ga Miteru 3rd,OVA


In [25]:
new_recs = pred_df.drop(filtered_df.loc[recommendee].anime_id, errors="ignore").loc[
    lambda x: (x["type"] != "Movie")
    & (x["type"] != "Special")
    & (x["type"] != "OVA")
    & (x["type"] != "ONA")
    & (x["type"] != "Music")
]

In [26]:
new_recs.loc[lambda x: (x["delta"] > 0)].sort_values(
    by="score_lower_bound", ascending=False
)[:30]

,delta,weight,counts,delta_sem,blp,score,sem,score_lower_bound,score_upper_bound,title,type
anime_id,,,,,,,,,,,
444,1.045192,1.302614,22,0.433283,6.766345,7.741370,0.419499,6.660813,8.821927,Maria-sama ga Miteru: Haru,TV
3750,1.090675,0.919299,15,0.534021,6.918957,7.936411,0.512538,6.616200,9.256622,Maria-sama ga Miteru 4th,TV
18195,0.271162,1.013641,21,0.391728,7.167813,7.420771,0.368475,6.471642,8.369900,Little Busters!: Refrain,TV
153,0.384402,1.829111,25,0.358492,6.915618,7.274214,0.338468,6.402377,8.146051,Juuni Kokuki,TV
3604,0.678634,1.877093,17,0.440808,6.776911,7.409986,0.418982,6.330760,8.489212,Hidamari Sketch x 365,TV
2164,0.028633,3.554515,35,0.293133,6.999254,7.025964,0.275011,6.317583,7.734346,Dennou Coil,TV
427,0.938484,0.586657,13,0.578555,6.850950,7.726430,0.550691,6.307944,9.144917,Kaleido Star,TV
232,0.485413,0.878325,16,0.457077,6.960820,7.413645,0.431494,6.302191,8.525100,Cardcaptor Sakura,TV
158,1.030665,1.383636,25,0.396764,6.323926,7.285399,0.386011,6.291101,8.279698,Maria-sama ga Miteru,TV


In [27]:
new_recs.loc[lambda x: x['title'].str.contains("Eien")]

,delta,weight,counts,delta_sem,blp,score,sem,score_lower_bound,score_upper_bound,title,type
anime_id,,,,,,,,,,,
147,-0.097016,0.995851,18,0.433378,6.26967,6.179167,0.406679,5.131631,7.226703,Kimi ga Nozomu Eien,TV


In [28]:
list(
    new_recs.loc[lambda x: (x["delta"] > 0)].sort_values(by="score_lower_bound", ascending=False)[:20][
        "title"
    ]
)

['Maria-sama ga Miteru: Haru',
 'Maria-sama ga Miteru 4th',
 'Little Busters!: Refrain',
 'Juuni Kokuki',
 'Hidamari Sketch x 365',
 'Dennou Coil',
 'Kaleido Star',
 'Cardcaptor Sakura',
 'Maria-sama ga Miteru',
 'Ichigo Mashimaro',
 'Senki Zesshou Symphogear AXZ',
 'Mugen no Ryvius',
 'Hidamari Sketch x ☆☆☆',
 'Kami nomi zo Shiru Sekai: Megami-hen',
 'Hidamari Sketch x Honeycomb',
 'Terra e... (TV)',
 'Yuru Camp△',
 'Sketchbook: Full Color&#039;s',
 'One Piece',
 'Toaru Kagaku no Railgun S']

In [32]:
a = ['Maria-sama ga Miteru: Haru',
 'Maria-sama ga Miteru 4th',
 'Little Busters!: Refrain',
 'Juuni Kokuki',
 'Hidamari Sketch x 365',
 'Dennou Coil',
 'Kaleido Star',
 'Cardcaptor Sakura',
 'Maria-sama ga Miteru',
 'Ichigo Mashimaro',
 'Senki Zesshou Symphogear AXZ',
 'Mugen no Ryvius',
 'Hidamari Sketch x ☆☆☆',
 'Kami nomi zo Shiru Sekai: Megami-hen',
 'Hidamari Sketch x Honeycomb',
 'Terra e... (TV)',
 'Yuru Camp△',
 'Sketchbook: Full Color&#039;s',
 'One Piece',
 'Toaru Kagaku no Railgun S']

In [33]:
b = ['Glass no Kamen (2005)',
 'New Game!!',
 'Eikoku Koi Monogatari Emma: Molders-hen',
 'Naruto',
 'Maria-sama ga Miteru',
 'Terra e... (TV)',
 'Hikaru no Go',
 'Mugen no Ryvius',
 'Naruto: Shippuuden',
 'Hidamari Sketch x 365',
 'Love Live! Sunshine!! 2nd Season',
 'Juuni Kokuki',
 'Hidamari Sketch x Honeycomb',
 'Hidamari Sketch x ☆☆☆',
 'Little Busters!: Refrain',
 'Hibike! Euphonium 2',
 'Kaleido Star',
 'One Piece',
 'Maria-sama ga Miteru: Haru',
 'Maria-sama ga Miteru 4th']

In [34]:
set(a) - set(b)

{'Cardcaptor Sakura',
 'Dennou Coil',
 'Ichigo Mashimaro',
 'Kami nomi zo Shiru Sekai: Megami-hen',
 'Senki Zesshou Symphogear AXZ',
 'Sketchbook: Full Color&#039;s',
 'Toaru Kagaku no Railgun S',
 'Yuru Camp△'}

In [35]:
set(b) - set(a)

{'Eikoku Koi Monogatari Emma: Molders-hen',
 'Glass no Kamen (2005)',
 'Hibike! Euphonium 2',
 'Hikaru no Go',
 'Love Live! Sunshine!! 2nd Season',
 'Naruto',
 'Naruto: Shippuuden',
 'New Game!!'}

In [ ]:
filtered_df.loc["Fro116"].to_pickle("user_profiles/normalized_Fro116.pkl")

In [ ]:
filtered_df.loc[lambda x: x['anime_id'].isin([10153, 24629, 17549, 77])].loc['Fro116']